In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os

# Load Models and Data (from TP5)

In [2]:
mlp_model = keras.models.load_model("mlp_model.h5")
cnn_model = keras.models.load_model("cnn_model.h5")

# Load the test dataset (we will use it as a representative dataset for quantization)

In [3]:
(_, _), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()


# Normalize the test data (same preprocessing as in TP5)

In [4]:
x_test_normalized = x_test.astype("float32") / 255.0

# Convert and Quantize the MLP Model

In [5]:
def representative_dataset_gen_mlp():
    for i in range(100): # 100 samples is usually enough
        # The model expects (28, 28) inputs
        yield [x_test_normalized[i:i+1]]

# Create the TFLiteConverter

In [6]:
converter_mlp = tf.lite.TFLiteConverter.from_keras_model(mlp_model)

# Enable quantization optimizations

In [8]:
converter_mlp.optimizations = [tf.lite.Optimize.DEFAULT]
converter_mlp.representative_dataset = representative_dataset_gen_mlp

# (Enforce full int8 quantization)

In [9]:
converter_mlp.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

# Convert the model

In [10]:
tflite_mlp_quant = converter_mlp.convert()

INFO:tensorflow:Assets written to: C:\Users\Rade\AppData\Local\Temp\tmpc95v78wm\assets


INFO:tensorflow:Assets written to: C:\Users\Rade\AppData\Local\Temp\tmpc95v78wm\assets


Saved artifact at 'C:\Users\Rade\AppData\Local\Temp\tmpc95v78wm'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  2589205324880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2589205325648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2589205323920: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2589205326608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2589205327568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2589205328144: TensorSpec(shape=(), dtype=tf.resource, name=None)


C:\Users\Rade\.conda\envs\env1\Lib\site-packages\tensorflow\lite\python\convert.py:863: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


# Save the .tflite file

In [11]:
with open("mlp_model_quant.tflite", "wb") as f:
    f.write(tflite_mlp_quant)

print("MLP model converted and saved as 'mlp_model_quant.tflite'")

MLP model converted and saved as 'mlp_model_quant.tflite'


# Convert and Quantize the CNN Model

# Prepare CNN data in the correct (N, H, W, C) shape

In [12]:

x_test_cnn = np.expand_dims(x_test_normalized, -1)

def representative_dataset_gen_cnn():
    for i in range(100):
        # The model expects (28, 28, 1) inputs
        yield [x_test_cnn[i:i+1]]



# Create the TFLiteConverter

In [13]:
converter_cnn = tf.lite.TFLiteConverter.from_keras_model(cnn_model)

# Enable quantization optimizations


In [14]:
converter_cnn.optimizations = [tf.lite.Optimize.DEFAULT]
converter_cnn.representative_dataset = representative_dataset_gen_cnn
converter_cnn.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

# Convert the model

In [15]:
tflite_cnn_quant = converter_cnn.convert()

INFO:tensorflow:Assets written to: C:\Users\Rade\AppData\Local\Temp\tmp2r43cjg3\assets


INFO:tensorflow:Assets written to: C:\Users\Rade\AppData\Local\Temp\tmp2r43cjg3\assets


Saved artifact at 'C:\Users\Rade\AppData\Local\Temp\tmp2r43cjg3'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='input_layer_3')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  2589205329488: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2589205330448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2589205331408: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2589205331792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2589205329296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2589205331024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2589205330640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2589205330064: TensorSpec(shape=(), dtype=tf.resource, name=None)


C:\Users\Rade\.conda\envs\env1\Lib\site-packages\tensorflow\lite\python\convert.py:863: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


# Save the .tflite file

In [16]:


with open("cnn_model_quant.tflite", "wb") as f:
    f.write(tflite_cnn_quant)

print("CNN model converted and saved as 'cnn_model_quant.tflite'")


CNN model converted and saved as 'cnn_model_quant.tflite'


# Print Final Results (for Task 4.1)

In [17]:
print("\n--- Task 4.1: Model Size Comparison Results ---")

# (Using the TP5 results you provided: 2.72MB & 0.69MB)
try:
    keras_mlp_size = os.path.getsize("mlp_model.h5") / (1024 * 1024)
    keras_cnn_size = os.path.getsize("cnn_model.h5") / (1024 * 1024)
except FileNotFoundError:
    print("Could not find .h5 files, using approximate sizes.")
    keras_mlp_size = 2.72 # (MB)
    keras_cnn_size = 0.69 # (MB)

# (Calculate the new TFLite sizes)
tflite_mlp_size_kb = os.path.getsize("mlp_model_quant.tflite") / 1024
tflite_cnn_size_kb = os.path.getsize("cnn_model_quant.tflite") / 1024

print(f"MLP (Keras): {keras_mlp_size:.2f} MB")
print(f"MLP (TFLite, int8): {tflite_mlp_size_kb:.2f} KB  <-- (Reduction!)")

print(f"\nCNN (Keras): {keras_cnn_size:.2f} MB")
print(f"CNN (TFLite, int8): {tflite_cnn_size_kb:.2f} KB  <-- (Reduction!)")



--- Task 4.1: Model Size Comparison Results ---
MLP (Keras): 2.72 MB
MLP (TFLite, int8): 243.62 KB  <-- (Reduction!)

CNN (Keras): 0.69 MB
CNN (TFLite, int8): 63.34 KB  <-- (Reduction!)
